In [6]:
#source: http://rasbt.github.io/mlxtend/user_guide/evaluate/mcnemar/#example-2-mcnemars-test-for-scenario-b
#comparing between lstm and the evolved neurons.

import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
import sys, os, math, time, datetime, re
import json, pickle

from mlxtend.evaluate import mcnemar_table, mcnemar

print(os.getcwd())
results_dir = '../../pythons/new_neurons/predict_results/20220809_har'
result_fileslist = [f for f in sorted(os.listdir(results_dir)) if os.path.isfile(os.path.join(results_dir, f))]

# importing the lstm predicted values (model 1)
model1_results = {}
for file_no in range(8):
    model1_results[file_no]  = np.array(pd.read_csv(os.path.join(results_dir, f'wisdm_lstm+_v00_{file_no}_predict.csv'), skiprows=None, header=None)).T[:,file_no*13769:]
    # print(f'model1_results[{file_no}] = {model1_results[file_no].shape}: {model1_results[file_no]}')

# importing the lstm target values (model 1)
targets = {}
for file_no in range(8):
    targets[file_no]  = np.array(pd.read_csv(os.path.join(results_dir, f'wisdm_lstm+_v00_{file_no}_target.csv'), skiprows=None, header=None)).T
    # print(f'targets[{file_no}] = {targets[file_no].shape}')

# importing the new neuron predicted values (model 1)
mcnemar_results = {}
p_values = {}
for best21_no in range(31):
    best21_version = f'0{best21_no}' if best21_no < 10 else f'{best21_no}'
    mcnemar_results[best21_version] = {}
    for file_no in range(8):
        mcnemar_results[best21_version][file_no] = {}
        model2_result = np.array(pd.read_csv(os.path.join(results_dir, f'wisdm_best21_v{best21_version}_{file_no}_predict.csv'), skiprows=None, header=None)).T[:,file_no*13769:]
        tb =  mcnemar_table(y_target=targets[file_no][0], 
                            y_model1=model1_results[file_no][0], 
                            y_model2=model2_result[0])
        b = int(tb[0][1])
        c = int(tb[1][0])
        if (b + c) > 25: #computing the corrected McNemar Test
            chi2, p = mcnemar(ary=tb, corrected=True)
        else: #computing the exact p-value from the binomial distribution
            chi2, p = mcnemar(ary=tb, exact=True)
        mcnemar_results[best21_version][file_no]['tb'] = tb
        mcnemar_results[best21_version][file_no]['chi-squared'] = chi2
        mcnemar_results[best21_version][file_no]['p-value'] = p
        print(f'best21_v{best21_version}_{file_no}_chi2_{chi2}_p_{p}')

# saving the results
with open('./mcnemar_results.pkl', 'wb') as f:
    pickle.dump(mcnemar_results, f)
    
with open('./mcnemar_results.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

# display(loaded_dict)

C:\Users\chaut\OneDrive - Heriot-Watt University\HIOF_Master\Master_Thesis\NewLSTM\Codes\tf_implementations\notebooks\rnnplus_for_har
best21_v00_0_ch2_80.36281588447653_p_3.11604123057337e-19
best21_v00_1_ch2_36.09423076923077_p_1.8800262499050523e-09
best21_v00_2_ch2_83.36381322957199_p_6.826016145972014e-20
best21_v00_3_ch2_63.001782531194294_p_2.06519597932329e-15
best21_v00_4_ch2_21.363449691991786_p_3.799458886223307e-06
best21_v00_5_ch2_35.378_p_2.7153672505976474e-09
best21_v00_6_ch2_109.13636363636364_p_1.5149663303862673e-25
best21_v00_7_ch2_67.17669902912621_p_2.482307102588045e-16
best21_v01_0_ch2_93.32854578096948_p_4.429571559781606e-22
best21_v01_1_ch2_44.01607142857143_p_3.256906674241216e-11
best21_v01_2_ch2_49.65045045045045_p_1.83727899794796e-12
best21_v01_3_ch2_46.637123745819395_p_8.54264597681378e-12
best21_v01_4_ch2_22.501893939393938_p_2.099365065160075e-06
best21_v01_5_ch2_14.529080675422138_p_0.0001380125777953704
best21_v01_6_ch2_91.07275803722504_p_1.3848519

In [3]:
#source: http://rasbt.github.io/mlxtend/user_guide/evaluate/mcnemar/#example-2-mcnemars-test-for-scenario-b
#comparing between the original evolved neurons and its manual modifications.

import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
import sys, os, math, time, datetime, re
import json, pickle

from mlxtend.evaluate import mcnemar_table, mcnemar

print(os.getcwd())
results_dir = '../../pythons/new_neurons/predict_results/20220809_har'
result_fileslist = [f for f in sorted(os.listdir(results_dir)) if os.path.isfile(os.path.join(results_dir, f))]

# importing the lstm predicted values (model 1)
model1_results = {}
for file_no in range(8):
    model1_results[file_no]  = np.array(pd.read_csv(os.path.join(results_dir, f'wisdm_best21_v00_{file_no}_predict.csv'), skiprows=None, header=None)).T[:,file_no*13769:]
    # print(f'model1_results[{file_no}] = {model1_results[file_no].shape}: {model1_results[file_no]}')

# importing the lstm target values (model 1)
targets = {}
for file_no in range(8):
    targets[file_no]  = np.array(pd.read_csv(os.path.join(results_dir, f'wisdm_best21_v00_{file_no}_target.csv'), skiprows=None, header=None)).T
    # print(f'targets[{file_no}] = {targets[file_no].shape}')

# importing the new neuron predicted values (model 1)
mcnemar_results = {}
p_values = {}
for best21_no in range(1, 31):
    best21_version = f'0{best21_no}' if best21_no < 10 else f'{best21_no}'
    mcnemar_results[best21_version] = {}
    for file_no in range(8):
        mcnemar_results[best21_version][file_no] = {}
        model2_result = np.array(pd.read_csv(os.path.join(results_dir, f'wisdm_best21_v{best21_version}_{file_no}_predict.csv'), skiprows=None, header=None)).T[:,file_no*13769:]
        tb =  mcnemar_table(y_target=targets[file_no][0], 
                            y_model1=model1_results[file_no][0], 
                            y_model2=model2_result[0])
        b = int(tb[0][1])
        c = int(tb[1][0])
        if (b + c) > 25: #computing the corrected McNemar Test
            chi2, p = mcnemar(ary=tb, corrected=True)
        else: #computing the exact p-value from the binomial distribution
            chi2, p = mcnemar(ary=tb, exact=True)
        mcnemar_results[best21_version][file_no]['tb'] = tb
        mcnemar_results[best21_version][file_no]['chi-squared'] = chi2
        mcnemar_results[best21_version][file_no]['p-value'] = p
        print(f'best21_v{best21_version}_{file_no}_chi2_{chi2}_p_{p}')

# saving the results
with open('./mcnemar_results_for_v00.pkl', 'wb') as f:
    pickle.dump(mcnemar_results, f)
    
with open('./mcnemar_results_for_v00.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

display(loaded_dict)

C:\Users\chaut\OneDrive - Heriot-Watt University\HIOF_Master\Master_Thesis\NewLSTM\Codes\tf_implementations\notebooks\rnnplus_for_har
best21_v01_0_chi2_0.7052341597796143_p_0.4010305099987682
best21_v01_1_chi2_0.8980099502487562_p_0.34331587762148963
best21_v01_2_chi2_4.092071611253197_p_0.04308480433750916
best21_v01_3_chi2_0.9876543209876543_p_0.3203163554920655
best21_v01_4_chi2_0.08933002481389578_p_0.7650306242707797
best21_v01_5_chi2_4.6650602409638555_p_0.030782343548976222
best21_v01_6_chi2_0.35910224438902744_p_0.549005249171516
best21_v01_7_chi2_1.065217391304348_p_0.3020282281097738
best21_v02_0_chi2_0.32180851063829785_p_0.5705228180175916
best21_v02_1_chi2_5.5248041775456915_p_0.01874870139248945
best21_v02_2_chi2_0.04456824512534819_p_0.8327999346701016
best21_v02_3_chi2_0.6181318681318682_p_0.43174234368243325
best21_v02_4_chi2_2.7457142857142856_p_0.09751549231682229
best21_v02_5_chi2_0.6859756097560976_p_0.4075366074605492
best21_v02_6_chi2_2.4523160762942777_p_0.11735

{'01': {0: {'tb': array([[13193,   173],
          [  190,   213]]),
   'chi-squared': 0.7052341597796143,
   'p-value': 0.4010305099987682},
  1: {'tb': array([[13138,   191],
          [  211,   229]]),
   'chi-squared': 0.8980099502487562,
   'p-value': 0.34331587762148963},
  2: {'tb': array([[13195,   216],
          [  175,   183]]),
   'chi-squared': 4.092071611253197,
   'p-value': 0.04308480433750916},
  3: {'tb': array([[13130,   213],
          [  192,   234]]),
   'chi-squared': 0.9876543209876543,
   'p-value': 0.3203163554920655},
  4: {'tb': array([[13144,   198],
          [  205,   222]]),
   'chi-squared': 0.08933002481389578,
   'p-value': 0.7650306242707797},
  5: {'tb': array([[13143,   230],
          [  185,   211]]),
   'chi-squared': 4.6650602409638555,
   'p-value': 0.030782343548976222},
  6: {'tb': array([[13195,   207],
          [  194,   173]]),
   'chi-squared': 0.35910224438902744,
   'p-value': 0.549005249171516},
  7: {'tb': array([[13149,   218],
   